<a href="https://colab.research.google.com/github/vfrantc/weather_experiments/blob/main/run_mprnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!git clone https://github.com/swz30/MPRNet.git

Cloning into 'MPRNet'...
remote: Enumerating objects: 321, done.
remote: Total 321 (delta 0), reused 0 (delta 0), pack-reused 321
Receiving objects: 100% (321/321), 87.83 KiB | 5.86 MiB/s, done.
Resolving deltas: 100% (171/171), done.


In [3]:
!pip install yacs joblib natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%cd MPRNet

/content/MPRNet


In [5]:
!cd pytorch-gradual-warmup-lr; python setup.py install; cd ..

running install
running bdist_egg
running egg_info
creating warmup_scheduler.egg-info
writing warmup_scheduler.egg-info/PKG-INFO
writing dependency_links to warmup_scheduler.egg-info/dependency_links.txt
writing top-level names to warmup_scheduler.egg-info/top_level.txt
writing manifest file 'warmup_scheduler.egg-info/SOURCES.txt'
writing manifest file 'warmup_scheduler.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/warmup_scheduler
copying warmup_scheduler/__init__.py -> build/lib/warmup_scheduler
copying warmup_scheduler/scheduler.py -> build/lib/warmup_scheduler
copying warmup_scheduler/run.py -> build/lib/warmup_scheduler
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/warmup_scheduler
copying build/lib/warmup_scheduler/__init__.py -> build/bdist.linux-x86_64/egg/warmup_scheduler
copying build/lib/warm

In [7]:
!cp /content/drive/MyDrive/derain/model_deraining.pth ./Deraining/pretrained_models

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
import os
from runpy import run_path
from skimage import img_as_ubyte
from collections import OrderedDict
from natsort import natsorted
import cv2
from glob import glob 
from tqdm.notebook import tqdm

In [9]:
def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [10]:
def load_checkpoint(model, weights):
    checkpoint = torch.load(weights)
    try:
        model.load_state_dict(checkpoint["state_dict"])
    except:
        state_dict = checkpoint["state_dict"]
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)

In [11]:
task = 'Deraining'

In [12]:
# Load corresponding model architecture and weights
load_file = run_path(os.path.join(task, "MPRNet.py"))
model = load_file['MPRNet']()
model.cuda()

MPRNet(
  (shallow_feat1): Sequential(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): ReLU(inplace=True)
          (2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): Sigmoid()
        )
      )
      (body): Sequential(
        (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): PReLU(num_parameters=1)
        (2): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
  )
  (shallow_feat2): Sequential(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(40, 

In [13]:
weights = os.path.join(task, "pretrained_models", "model_" + task.lower()+".pth")
load_checkpoint(model, weights)
model.eval()

MPRNet(
  (shallow_feat1): Sequential(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): ReLU(inplace=True)
          (2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): Sigmoid()
        )
      )
      (body): Sequential(
        (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): PReLU(num_parameters=1)
        (2): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
  )
  (shallow_feat2): Sequential(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): CAB(
      (CA): CALayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (conv_du): Sequential(
          (0): Conv2d(40, 

In [14]:
img_multiple_of = 8

In [15]:
def restore_frame(img):
    input_ = TF.to_tensor(img).unsqueeze(0).cuda()

    # Pad the input if not_multiple_of 8
    h,w = input_.shape[2], input_.shape[3]
    H,W = ((h+img_multiple_of)//img_multiple_of)*img_multiple_of, ((w+img_multiple_of)//img_multiple_of)*img_multiple_of
    padh = H-h if h%img_multiple_of!=0 else 0
    padw = W-w if w%img_multiple_of!=0 else 0
    input_ = F.pad(input_, (0,padw,0,padh), 'reflect')

    with torch.no_grad():
        restored = model(input_)
    restored = restored[0]
    restored = torch.clamp(restored, 0, 1)

    # Unpad the output
    restored = restored[:,:,:h,:w]

    restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
    restored = img_as_ubyte(restored[0])
    return restored 

# Process derain datasets

In [17]:
!cp /content/drive/MyDrive/weather_experiments/weather_test.zip .

In [18]:
!unzip weather_test.zip 

Streaming output truncated to the last 5000 lines.
  inflating: test/outdoor/input/0194_1_0.2.jpg  
  inflating: test/outdoor/input/1986_0.9_0.08.jpg  
  inflating: test/outdoor/input/0251_0.8_0.16.jpg  
  inflating: test/outdoor/input/1953_0.8_0.2.jpg  
  inflating: test/outdoor/input/0127_0.8_0.2.jpg  
  inflating: test/outdoor/input/0003_0.8_0.2.jpg  
  inflating: test/outdoor/input/1861_0.85_0.2.jpg  
  inflating: test/outdoor/input/0317_0.85_0.08.jpg  
  inflating: test/outdoor/input/0269_0.95_0.2.jpg  
  inflating: test/outdoor/input/1055_0.95_0.2.jpg  
  inflating: test/outdoor/input/0267_0.9_0.16.jpg  
  inflating: test/outdoor/input/1848_0.8_0.12.jpg  
  inflating: test/outdoor/input/0082_0.85_0.16.jpg  
  inflating: test/outdoor/input/0138_0.9_0.08.jpg  
  inflating: test/outdoor/input/1724_0.95_0.2.jpg  
  inflating: test/outdoor/input/1889_0.85_0.08.jpg  
  inflating: test/outdoor/input/0302_0.9_0.16.jpg  
  inflating: test/outdoor/input/1868_1_0.08.jpg  
  inflating: test/

In [26]:
def process_one(input_dir, output_dir):
  print(f'{input_dir} ----> {output_dir}')
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  for fname in tqdm(list(glob(os.path.join(input_dir, '*.jpg'))) + list(glob(os.path.join(input_dir, '*.png')))):
    image = cv2.imread(fname, 1)
    image = restore_frame(image)
    cv2.imwrite(os.path.join(output_dir, os.path.basename(fname)), image)

In [24]:
!ls test/real/input/

001.png  076.png  149.png  222.png  286.png  349.png  413.png  561.png	618.png
002.png  078.png  155.png  223.png  289.png  350.png  414.png  562.png	619.png
003.png  079.png  156.png  224.png  290.png  351.png  415.png  563.png	621.png
004.png  080.png  158.png  225.png  291.png  352.png  425.png  564.png	622.png
005.png  081.png  159.png  226.png  292.png  353.png  437.png  565.png	623.png
006.png  082.png  160.png  227.png  293.png  354.png  467.png  566.png	624.png
007.png  083.png  161.png  228.png  295.png  355.png  469.png  567.png	625.png
008.png  084.png  162.png  229.png  296.png  356.png  474.png  568.png	626.png
009.png  087.png  163.png  230.png  297.png  357.png  475.png  569.png	627.png
010.png  088.png  165.png  231.png  298.png  358.png  476.png  570.png	628.png
011.png  089.png  166.png  232.png  299.png  359.png  477.png  571.png	629.png
012.png  091.png  167.png  233.png  300.png  360.png  478.png  572.png	630.png
013.png  092.png  168.png  235.png  301.png  362.png

In [22]:
process_one('test/rain1400/input/', 'mprnet/rain1400')
process_one('test/rain/input/', 'mprnet/rain')


test/rain1400/input/ ----> mprnet/rain1400


  0%|          | 0/2800 [00:00<?, ?it/s]

test/rain/input/ ----> mprnet/rain


  0%|          | 0/400 [00:00<?, ?it/s]

test/real/input/ ----> mprnet/real


0it [00:00, ?it/s]

In [27]:
process_one('test/real/input/', 'mprnet/real')

test/real/input/ ----> mprnet/real


  0%|          | 0/497 [00:00<?, ?it/s]

In [37]:
!ls test/test1/input

im_0301_s100_a04.png  im_0317_s90_a05.png   im_0334_s80_a06.png
im_0301_s100_a05.png  im_0317_s90_a06.png   im_0334_s85_a04.png
im_0301_s100_a06.png  im_0317_s95_a04.png   im_0334_s85_a05.png
im_0301_s80_a04.png   im_0317_s95_a05.png   im_0334_s85_a06.png
im_0301_s80_a05.png   im_0317_s95_a06.png   im_0334_s90_a04.png
im_0301_s80_a06.png   im_0318_s100_a04.png  im_0334_s90_a05.png
im_0301_s85_a04.png   im_0318_s100_a05.png  im_0334_s90_a06.png
im_0301_s85_a05.png   im_0318_s100_a06.png  im_0334_s95_a04.png
im_0301_s85_a06.png   im_0318_s80_a04.png   im_0334_s95_a05.png
im_0301_s90_a04.png   im_0318_s80_a05.png   im_0334_s95_a06.png
im_0301_s90_a05.png   im_0318_s80_a06.png   im_0335_s100_a04.png
im_0301_s90_a06.png   im_0318_s85_a04.png   im_0335_s100_a05.png
im_0301_s95_a04.png   im_0318_s85_a05.png   im_0335_s100_a06.png
im_0301_s95_a05.png   im_0318_s85_a06.png   im_0335_s80_a04.png
im_0301_s95_a06.png   im_0318_s90_a04.png   im_0335_s80_a05.png
im_0302_s100_a04.png  im_0318_s90_a05

In [38]:
process_one('test/test1/input/', 'mprnet/test1')

test/test1/input/ ----> mprnet/test1


  0%|          | 0/750 [00:00<?, ?it/s]

In [33]:
!ls mprnet/rain1400

1000_10.jpg  916_6.jpg	 933_2.jpg   950_12.jpg  966_8.jpg   983_4.jpg
1000_11.jpg  916_7.jpg	 933_3.jpg   950_13.jpg  966_9.jpg   983_5.jpg
1000_12.jpg  916_8.jpg	 933_4.jpg   950_14.jpg  967_10.jpg  983_6.jpg
1000_13.jpg  916_9.jpg	 933_5.jpg   950_1.jpg	 967_11.jpg  983_7.jpg
1000_14.jpg  917_10.jpg  933_6.jpg   950_2.jpg	 967_12.jpg  983_8.jpg
1000_1.jpg   917_11.jpg  933_7.jpg   950_3.jpg	 967_13.jpg  983_9.jpg
1000_2.jpg   917_12.jpg  933_8.jpg   950_4.jpg	 967_14.jpg  984_10.jpg
1000_3.jpg   917_13.jpg  933_9.jpg   950_5.jpg	 967_1.jpg   984_11.jpg
1000_4.jpg   917_14.jpg  934_10.jpg  950_6.jpg	 967_2.jpg   984_12.jpg
1000_5.jpg   917_1.jpg	 934_11.jpg  950_7.jpg	 967_3.jpg   984_13.jpg
1000_6.jpg   917_2.jpg	 934_12.jpg  950_8.jpg	 967_4.jpg   984_14.jpg
1000_7.jpg   917_3.jpg	 934_13.jpg  950_9.jpg	 967_5.jpg   984_1.jpg
1000_8.jpg   917_4.jpg	 934_14.jpg  951_10.jpg  967_6.jpg   984_2.jpg
1000_9.jpg   917_5.jpg	 934_1.jpg   951_11.jpg  967_7.jpg   984_3.jpg
901_10.jpg   917_6.

In [40]:
!zip -r mprnet_rain.zip mprnet/


updating: mprnet/ (stored 0%)
  adding: mprnet/real/ (stored 0%)
  adding: mprnet/real/195.png (deflated 1%)
  adding: mprnet/real/663.png (deflated 3%)
  adding: mprnet/real/169.png (deflated 1%)
  adding: mprnet/real/367.png (deflated 1%)
  adding: mprnet/real/260.png (deflated 4%)
  adding: mprnet/real/559.png (deflated 2%)
  adding: mprnet/real/271.png (deflated 0%)
  adding: mprnet/real/003.png (deflated 2%)
  adding: mprnet/real/008.png (deflated 2%)
  adding: mprnet/real/113.png (deflated 3%)
  adding: mprnet/real/135.png (deflated 2%)
  adding: mprnet/real/631.png (deflated 2%)
  adding: mprnet/real/290.png (deflated 2%)
  adding: mprnet/real/257.png (deflated 3%)
  adding: mprnet/real/469.png (deflated 2%)
  adding: mprnet/real/350.png (deflated 2%)
  adding: mprnet/real/495.png (deflated 2%)
  adding: mprnet/real/214.png (deflated 2%)
  adding: mprnet/real/591.png (deflated 3%)
  adding: mprnet/real/606.png (deflated 1%)
  adding: mprnet/real/314.png (deflated 2%)
  adding: m

In [41]:
!cp mprnet_rain.zip /content/drive/MyDrive/weather_experiments/